# BERT for BETO

In [1]:
import pandas as pd
import numpy as np
import torch
import random

In [47]:
#Load in some data
save_path = '/Users/Jonathan/Desktop/LabeledChemEData/Labeled_Sheets/'
#Do we have something that allows us to fill empty cells with "" or something else? Maybe set any NaNs to 0? That would imply
#A null label. 
df = pd.read_excel(save_path+"Carbon_0.xlsx")

In [50]:
df

,Unnamed: 0,name,tokens,BESIO,entity,mol_class,Unnamed: 6,name.1,tokens.1,BESIO.1,...,tokens.48,BESIO.48,entity.48,mol_class.48,Unnamed: 294,name.49,tokens.49,BESIO.49,entity.49,mol_class.49
0,0,Jon O,In,NaN,NaN,NaN,0,Jon O,X-ray,NaN,...,©,NaN,NaN,NaN,0,Jon O,©,NaN,NaN,NaN
1,1,2010,the,NaN,NaN,NaN,1,2014,photoelectron,NaN,...,2020,NaN,NaN,NaN,1,2015,2015,NaN,NaN,NaN
2,2,250,interaction,NaN,NaN,NaN,2,114,spectroscopy,NaN,...,Elsevier,NaN,NaN,NaN,2,104,Elsevier,NaN,NaN,NaN
3,3,10.1016/j.carbon.2010.02.003,between,NaN,NaN,NaN,3,10.1016/j.carbon.2013.12.061,(XPS),NaN,...,Ltd,NaN,NaN,NaN,3,10.1016/j.carbon.2015.08.007,Ltd.,NaN,NaN,NaN
4,4,NaN,gas,NaN,NaN,NaN,4,NaN,has,NaN,...,The,NaN,NaN,NaN,4,NaN,Hydrogels,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,349,NaN,NaN,NaN,NaN,NaN,349,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,349,NaN,NaN,NaN,NaN,NaN
350,350,NaN,NaN,NaN,NaN,NaN,350,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,350,NaN,NaN,NaN,NaN,NaN
351,351,NaN,NaN,NaN,NaN,NaN,351,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,351,NaN,NaN,NaN,NaN,NaN
352,352,NaN,NaN,NaN,NaN,NaN,352,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,352,NaN,NaN,NaN,NaN,NaN


Need to update extract_xy_ to pull out just the x values (ie, tokens), and the y values as a smashed together BESIO. He used NLTK sent_tokenize, so we are in the same place, and we can just go back to having sentences or something.

In [96]:
def extract_xy_(df):
    """
    This method extracts and correctly aranges the NER training x-values (tokens)
    and y-values (BESIO labels) from a pandas dataframe containing labeled NER
    data

    Parameters:
        df (pandas DataFrame, required): Dataframe loaded via pd.read_excel() on
            a labeled NER dataset

        endings_dict (dictionary, required): Dictionary containing the indicies
            where each sentence in each line of tokens ends.

    Returns:
        list: List of tuples, containing the x,y pairs
    """
    labeled = []
    columns = df.columns
    new_df = pd.DataFrame()
    all_tokens = []             # will contain lists of tokens, one for each text

    besio = []
    mol = []
    IorO = []
        
    for idx, column in enumerate(columns):
        # find every column that starts with 'name'
        if column.startswith('name'):

            # check if the entry in 'name' cell is a str
            if isinstance(df[column][0], str):
                tokens = df[columns[idx + 1]].values
                all_tokens.append(tokens)
                #If data is labeled, we should collect them into lists.
#                 if nextcol.startswith('BESIO'):
#                     labels = df[nextcol].replace(np.nan, 'O')
#                 else:
#                     labels = df[nextcol].replace(np.nan, '')
                df[columns[idx+2]].replace(np.nan, 'O', inplace = True)
                besio.append(df[columns[idx+2]].values)
                df[columns[idx+3]].replace(np.nan, '', inplace = True)
                mol.append(df[columns[idx+3]].values)
                df[columns[idx+4]].replace(np.nan, '', inplace = True)
                IorO.append(df[columns[idx+4]].values)

    i = 0

    label_values = []
   # print(len(besio))
    while i < len(besio):
        label_values.append([])
       # print(len(besio[i]))
       # print(range(len(besio[i])))
        for j in range(len(besio[i])):
            if besio[i][j] == 'O':
                label_values[i].append(besio[i][j])
            else:
                label_values[i].append(besio[i][j]+'-'+mol[i][j]+'-'+IorO[i][j])
        i += 1   
    return all_tokens, label_values


In [97]:
tokens, labels = extract_xy_(df)

In [98]:
labels[1]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-PRO-',
 'O',
 'O',
 'O',
 'S-MOL-I',
 'O',
 'O',
 'O',
 'O',
 'B-PRO-',
 'E-PRO-',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PRO-',
 'E-PRO-',
 'O',
 'O',
 'B-PRO-',
 'E-PRO-',
 'O',
 'B-PRO-',
 'E-PRO-',
 'O',
 'O',
 'B-PRO-',
 'I-PRO-',
 'I-PRO-',
 'I-PRO-',
 'E-PRO-',
 'S-PRO-',
 'O',
 'O',
 'O',
 'B-PRO-',
 'E-PRO-',
 'O',
 'S-PRO-',
 'O',
 'S-PRO-',
 'O',
 'S-PRO-',
 'O',
 'O',
 'O',
 'S-MOL-I',
 'O',
 'S-PRO-',
 'O',
 'O',
 'O',
 'O',
 'S-MOL-O',
 'S-MOL-O',
 'S-MOL-O',
 'B-MOL-O',
 'E-MOL-O',
 'S-MOL-O',
 'S-MOL-O',
 'S-MOL-O',
 'S-MOL-O',
 'S-MOL-O',
 'S-MOL-O',
 'O',
 'B-MOL-O',
 'I-MOL-O',
 'E-MOL-O',
 'S-MOL-O',
 'O',
 'O',
 'O',
 'S-MOL-O',
 'O',
 'S-MOL-O',
 'O',
 'O',
 'S-MOL-O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-PRO-',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-PRO-',
 'O',
 'S-PRO-',
 'O',
 'O',
 'O',
 'S-PRO-',
 'O',
 'O',

In [88]:
tokens[1]

array(['X-ray', 'photoelectron', 'spectroscopy', '(XPS)', 'has', 'been',
       'commonly', 'used', 'to', 'determine', 'the',
       'nitrogen-containing', 'functional', 'groups', 'of', 'graphene.',
       'However,', 'reported', 'assignments', 'of', 'C1s', 'shifts', 'of',
       'nitrogen-containing', 'functional', 'groups', 'are', 'unclear.',
       'Most', 'works', 'discuss', 'peak', 'shifts', 'of', 'only', 'N1s',
       'spectra', 'and', 'C1s', 'shifts', 'and', 'the', 'full', 'width',
       'at', 'half', 'maximum', '(FWHM)', 'are', 'excluded.', 'Thus,',
       'peak', 'shifts', 'and', 'FWHMs', 'of', 'C1s', 'and', 'N1s', 'XPS',
       'spectra', 'of', 'graphene', 'with', 'nitrogen-containing',
       'functional', 'groups', 'such', 'as', 'pyridinic,',
       'phenanthroline-like,', 'sp2C-NH2,', 'sp', '3C-NH2,', 'pyrrolic,',
       'imine,', 'pyridazine-like,', 'pyrazole-like,', 'sp2C-CN,',
       'sp3C-CN,', 'and', 'valley', 'quaternary', 'nitrogen', '(Q-N)',
       'on', 'edges', 

Ok, so now we have two lists. Both lists are structured so that each entry represents a unique paper (ie, tokens[1] is a whole paper). We need to now chop it down so that all the extra entries at the end are removed, and the list tokens[1] is only as long as there are words in that paper. 

Once we have done that for each entry, we will need to build a looping/wrapper function that will read every excel sheet in a directory. It'd be ideal if that looping function would append each new list to the previous list, so we could end up with every single labeled paper in a single set of two lists. 

After we have that function built, the next step is to try to regenerate our sentence-split structure. First step is to make each paper back into a single string. We'll do this by making a homemade inverse .split() function, which means we'll add each item in the tokens[1] list together with a single whitespace between them. Example is in the case ['The', 'dog', 'ran.'] we would want to regen the original string of ['The dog ran.']. We could do that by doing original_string += (token[1][i]+ ' '). Once we have made each paper in the list back into individual strings, we'll chop each string into individual sentences by using NLTK. 

From that point, it's more standard BERT. We'll use BERT's tokenizer. We'll need to make sure we hand-extend each label to match the tokenization done by the BERT tokenizer so we don't have length mismatches (a la: https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/ function tokenize_and_preserve_labels) and then we'll send it to a dataloader. 

In [ ]:
#Wrapper function that loops all excel spreadsheets and truncates each entry in each spreadsheet.